# ResNet

![](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c1/Piramidal_cell.svg/440px-Piramidal_cell.svg.png)

>ResNet consists of residual blocks that transfer the knowledge from one layer to  further layers by skipping some layers in between. These kinds of connections of  layers are known as skip-connections since we are skipping one or more layers.  Skip-connections help with the vanishing gradient issue by propagating the  gradients to further layers. This allows us to train very large convolutional neural  networks without loss of performance.
>Thakur, Abhishek. Approaching (Almost) Any Machine Learning Problem (p.205). Abhishek Thakur. 

The model pipeline is derived from this amazing notebook, so please upvote it too!

[OWN Jane Street with Keras NN](https://www.kaggle.com/tarlannazarov/own-jane-street-with-keras-nn)

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices


SEED = 1111

tf.random.set_seed(SEED)
np.random.seed(SEED)

# train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = pd.read_feather('../input/janestreet-save-as-feather/train.feather')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

train.fillna(train.mean(),inplace=True)

train['action'] = ((train['resp'].values) > 0).astype(int)


features = [c for c in train.columns if "feature" in c]

f_mean = np.mean(train[features[1:]].values,axis=0)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

In [ ]:
# split features for a ResNet
features1 = [f'feature_{i}' for i in range(71)]
features2 = [f'feature_{i}' for i in range(71, 130)]

In [ ]:
X_train = [train.loc[:, features1].values, 
          train.loc[:, features2].values]

In [ ]:
assert len(features) == len(features1) + len(features2)

# ResNet
derived from [[janestreet] ResNet with AutoEncoder (infer)](https://www.kaggle.com/code1110/janestreet-resnet-with-autoencoder-infer).

In [ ]:
def create_resnet(n_features, n_features_2, n_labels, learning_rate=1e-03, label_smoothing=1e-02):    
    input_1 = tf.keras.layers.Input(shape = (n_features,), name = 'Input1')
    input_2 = tf.keras.layers.Input(shape = (n_features_2,), name = 'Input2')

    head_1 = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(256, activation="elu"), 
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(256, activation = "elu")
        ],name='Head1') 

    input_3 = head_1(input_1)
    input_3_concat = tf.keras.layers.Concatenate()([input_2, input_3])

    head_2 = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(256, "elu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(256, "relu"),
        ],name='Head2')

    input_4 = head_2(input_3_concat)
    input_4_avg = tf.keras.layers.Average()([input_3, input_4]) 

    head_3 = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, kernel_initializer='lecun_normal', activation='selu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(n_labels, kernel_initializer='lecun_normal', activation='selu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(n_labels, activation="sigmoid")
        ],name='Head3')

    output = head_3(input_4_avg)

    model = tf.keras.models.Model(inputs = [input_1, input_2], outputs = output)
    opt = tfa.optimizers.RectifiedAdam(learning_rate=learning_rate)
    opt = tfa.optimizers.SWA(opt)
    model.compile(optimizer=opt, 
                  loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing), 
                  metrics=['AUC'])
    
    return model

tf.keras.backend.clear_session()
clf = create_resnet(len(features1), len(features2), len(resp_cols), 1e-03, 1e-02)

clf.summary()

In [ ]:
tf.keras.utils.plot_model(clf)

# Fit & Predict

In [ ]:

clf.fit(X_train, y_train, epochs=200, batch_size=4096 * 16)

# save model
clf.save('resnet.h5')

models = [clf]

th = 0.501

f = np.median
models = models[-3:]
import janestreet
env = janestreet.make_env()
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = np.mean([model([x_tt[:, :len(features1)], x_tt[:, len(features1):]], training = False).numpy() for model in models],axis=0)
        pred = f(pred)
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

That's it!